In [15]:
import random, uuid, datetime, json, sys
def get_random_words(words_list, total=1):
    ran_words = []

    # enumerate over specified number of words
    while len(ran_words) < total:
        ran_words += [words_list[random.randint(0, len(words_list)-1)]]

    # return a string by joining the list of words
    return ' '.join(ran_words)

def get_random_date(min_year=1990, max_year=datetime.datetime.now().year):
    #generate a datetime in format yyyy-mm-dd",
    start = datetime.datetime(min_year,1,1)
    years = max_year - min_year + 1
    end = start + datetime.timedelta(days=365 * years)
    return (start + (end - start) * random.random()).strftime('%Y-%m-%d')

def create_postgres_json(size):
    # list to store JSON records
    records = []

    # random words to inject into records
    words = ["apps name", "Facebook", "Pokémon GO", 'Google', 'examples', 'foo', 'bar']
    # iterate over the number of records being created
    for rec_id in range(size):

        # create a new record dict
        new_record = {}

        # input a value for each table column
        new_record['pk'] = rec_id
        new_record['id'] = uuid.uuid1().hex
        new_record['title'] = words[random.randint(0, len(words)-1)]
        new_record['description'] = get_random_words(words, random.randint(1, len(words)))
        new_record['published_timestamp'] = get_random_date()
        new_record['last_update_timestamp'] = get_random_date(int(new_record['published_timestamp'].split("-")[0]))

        # append the new record dict to the list
        records += [ new_record ]

    # return the list of JSON records
    return records


In [16]:
json_records = create_postgres_json(5000000)
json_records_str = json.dumps(json_records, indent=4)

In [17]:
#Upload random generated data to Postgresql.

dbname = '<YOUR DBNAME>'
user = '<YOUR USER>'
host ='<YOUR HOST>'
password = '<YOUR PASSWORD>'
table_name = 'apps'

from psycopg2 import sql, Error, connect
def create_insert_records(json_array, table_name):

    # get the columns for the JSON records
    columns = json_array[0].keys()

    # SQL column names should be lowercase using underscores instead of spaces/hyphens
    columns = [str(col).lower().replace(" ", "_") for col in columns]
    columns = [str(col).lower().replace("-", "_") for col in columns]

    # concatenate a string for the SQL 'INSERT INTO' statement
    sql_string = "INSERT INTO {}".format(table_name)
    sql_string = sql_string + " (" + ', '.join(columns) + ")\nVALUES "

    record_list = []
    for i, record in enumerate(json_array):

        keys = record.keys()
        values = record.values()

        # use map() to cast all items in record list as a string
        #record = list(map(str, values))
        record = list(values)

        # fix the values in the list if needed
        for i, val in enumerate(record):

            if type(val) == str:
                if "'" in val:
                    # posix escape string syntax for single quotes
                    record[i] = "E'" + record[i].replace("'", "''") + "'"

        # cast record as string and remove the list brackets []
        record = str(record).replace("[", '')
        record = record.replace("]", '')

        # remove double quotes as well
        record = record.replace('"', '')

        # ..now append the records to the list
        record_list += [ record ]

    # enumerate() over the records and append to SQL string
    for i, record in enumerate(record_list):

        # use map() to cast all items in record list as a string
        #record = list(map(str, record))

        # append the record list of string values to the SQL string
        sql_string = sql_string + "(" + record + "),\n"

    # replace the last comma with a semicolon
    sql_string = sql_string[:-2] + ";"

    return sql_string




# call the function to create INSERT INTO SQL string
sql_str = create_insert_records(json_records, table_name)

try:
    # declare a new PostgreSQL connection object
    conn = connect(
        dbname = dbname,
        user = user,
        host = host,
        password = password,
        # attempt to connect for 3 seconds then raise exception
        connect_timeout = 3
    )

    cur = conn.cursor()
    print ("\ncreated cursor object:", cur)

except Error as err:
    print ("\npsycopg2 connect error:", err)
    conn = None
    cur = None
    
# only attempt to execute SQL if cursor is valid
if cur != None:

    try:
        sql_resp = cur.execute(sql_str)
        conn.commit()
        print ('finished INSERT INTO execution')

    except (Exception, Error) as error:
        print("\nexecute_sql() error:", error)
        conn.rollback()

    # close the cursor and connection
    cur.close()
    conn.close()


created cursor object: <cursor object at 0x000001C22127F3C8; closed: 0>
finished INSERT INTO execution
